# _Moneyballing Fantasy Premier League Football_
# Dataset Creation

Having gathered all the raw data, and constructed some of the tables that we'll need, we now need to create a dataset that we can actually feed into a model. In particular, we will need functions to create two things:

* A full historic dataset that we can use to train models. 
* A dataset for a given week, that a model can use to make its predictions.

### Bringing in the required libraries...

In [1]:
#Standard data manipulations
import pandas as pd
import numpy as np
from datetime import datetime, date
import time

#SQL
import sqlite3
#Including custom functions, stored elsewhere in the repo
from PythonFunctions.sqlfunctions import *

#Pickle!
import pickle

#Suppress warnings from showing
import warnings
warnings.filterwarnings('ignore')

## Fixtures Table

Let's bring in the fixtures table we made in the previous notebook.

In [2]:
#Create the database 'fpl.db' (fantasy premier league!)
conn = sqlite3.connect('Data/fpl.db')
#Instantiate a cursor
c = conn.cursor()

In [3]:
df_fixtures = sql('select * from FixturesBasic', c).drop('TableIndex', axis=1)

#Change dates to datetime objects where possible...
df_fixtures['Date'] = pd.to_datetime(df_fixtures['Date'], errors='coerce')
df_fixtures.replace('AFC Bournemouth', 'Bournemouth', inplace=True)
df_fixtures['GameWeek'] = pd.to_numeric(df_fixtures['GameWeek'])

df_fixtures.head()

,MatchID,GameWeek,Date,HomeTeam,AwayTeam
FixturesBasicID,,,,,
1,46605,1,2019-08-09,Liverpool,Norwich City
2,46606,1,2019-08-10,Bournemouth,Sheffield United
3,46607,1,2019-08-10,Burnley,Southampton
4,46608,1,2019-08-10,Crystal Palace,Everton
5,46609,1,2019-08-11,Leicester City,Wolverhampton Wanderers


## Creating datasets

Remember, the aim here was to create datasets to put into our models (first to train, then to predict). That data will look something like this:

For a given player, consider their, and their opponent's matches such that we can get data for:
* Their last match
* Their last X matches
* Their last Y home/away matches as required

To that end, we need some functionality that will tell us which matches fit certain criteria. Let's create a melted dataframe from the fixtures df that has only one team per row.

In [4]:
df_home = df_fixtures.drop('AwayTeam', axis=1)
df_away = df_fixtures.drop('HomeTeam', axis=1)

df_home['Home']=df_home['MatchID'].map(lambda x: 'Home')
df_away['Home']=df_away['MatchID'].map(lambda x: 'Away')

df_away

cols = ['MatchID','GameWeek','Date','Team','Home']
df_home.columns = cols
df_away.columns = cols

#Combine the above dataframes, sorting by matchid
df_melt = pd.concat([df_home,df_away])
df_melt.sort_values(['MatchID','Home'], ascending=[True, False], inplace=True)

AgainstTeam = []
#Get the opposing team
for row in df_melt.itertuples():
    #For each row get the match ID
    match = row.MatchID
    #And the required team
    home = row.Home
    if home == 'Away':
        home = 'Home'
    else:
        home = 'Away'
    home = f'{home}Team'
    requiredTeam = df_fixtures.loc[(df_fixtures['MatchID']==match),home].item()
    AgainstTeam.append(requiredTeam)
    
df_melt['AgainstTeam'] = AgainstTeam
df_melt.reset_index(inplace=True, drop=True)

df_melt.head()

,MatchID,GameWeek,Date,Team,Home,AgainstTeam
0,46605,1,2019-08-09,Liverpool,Home,Norwich City
1,46605,1,2019-08-09,Norwich City,Away,Liverpool
2,46606,1,2019-08-10,Bournemouth,Home,Sheffield United
3,46606,1,2019-08-10,Sheffield United,Away,Bournemouth
4,46607,1,2019-08-10,Burnley,Home,Southampton


Now let's bring in the data from the SQL database.

In [5]:
#Print the names of the tables in the SQL database
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

TeamsBasic
PlayerAPIStats
PlayersBasic
ShotsDetail
PlayerMatchesDetail
TeamMatchesDetail
FixturesBasic
TeamsXGC
PlayersXG


In [6]:
df_matches_p = sql('select * from PlayerMatchesDetail', c).drop('TableIndex', axis=1)
df_matches_t = sql('select * from TeamMatchesDetail', c).drop('TableIndex', axis=1)

Let's remind ourselves what these look like...

In [7]:
df_matches_p.head(3)

,Player,GameWeek,Minutes,ForTeam,AgainstTeam,RelativeStrength,Goals,ShotsOnTarget,ShotsInBox,CloseShots,...,ShotsRight,GoalAssists,ShotOnTargetCreated,ShotInBoxCreated,CloseShotCreated,TotalShotCreated,HeadersCreated,CreatedCentre,CreatedLeft,CreatedRight
PlayerMatchesDetailID,,,,,,,,,,,,,,,,,,,,,
1,Alisson,1,90,Liverpool,Norwich City,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Virgil van Dijk,1,90,Liverpool,Norwich City,3,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Joseph Gomez,1,90,Liverpool,Norwich City,3,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
df_matches_t.head(3)

,MatchID,ForTeam,AgainstTeam,RelativeStrength,GameWeek,Home,Possession,Goals,ShotsOnTarget,TotalShots,...,Headers,ShotsCentre,ShotsLeft,ShotsRight,ShotsInBoxConceded,CloseShotsConceded,HeadersConceded,ShotsCentreConceded,ShotsLeftConceded,ShotsRightConceded
TeamMatchesDetailID,,,,,,,,,,,,,,,,,,,,,
1,46605,Liverpool,Norwich City,3,1,Home,57.9,4,7,15,...,2,6,2,2,7,7,1,4,3,1
2,46605,Norwich City,Liverpool,-3,1,Away,42.1,1,5,12,...,1,4,3,1,10,9,2,6,2,2
3,46606,Bournemouth,Sheffield United,0,1,Home,52.9,1,3,13,...,1,4,1,1,2,4,0,2,0,0


We could do with having the match ID and home/away in the player matches dataframe...

In [9]:
#Instantiate an empty list
match_ids = []
home_away = []
dates = []

#For each row in the player matches dataframe...
for row in df_matches_p.itertuples():
    #Look up the match id from the team matches dataframe
    team = row.ForTeam
    againstteam = row.AgainstTeam
    gameweek = row.GameWeek

    match_id = df_melt.loc[(df_melt['GameWeek']==gameweek)
                          &(df_melt['Team']==team)
                          &(df_melt['AgainstTeam']==againstteam),
                          'MatchID'].item()
    
    date = df_melt.loc[(df_melt['GameWeek']==gameweek)
                          &(df_melt['Team']==team)
                          &(df_melt['AgainstTeam']==againstteam),
                          'Date'].item()
    
    home = df_melt.loc[(df_melt['GameWeek']==gameweek)
                          &(df_melt['Team']==team)
                          &(df_melt['AgainstTeam']==againstteam),
                          'Home'].item()
    
    #Add it to the list
    match_ids.append(match_id)
    home_away.append(home)
    dates.append(date)
    
#Declare the list as a column in the player matches df
df_matches_p['MatchID']=match_ids
df_matches_p['Date']=pd.to_datetime(dates)
df_matches_p['Home']=home_away

#Get the match IDs column first in the dataframe
cols = list(df_matches_p.columns)
new_cols = ['MatchID', 'Date', 'Home'] + cols[:-3]
df_matches_p = df_matches_p[new_cols]

In [10]:
df_matches_p.head(3)

,MatchID,Date,Home,Player,GameWeek,Minutes,ForTeam,AgainstTeam,RelativeStrength,Goals,...,ShotsRight,GoalAssists,ShotOnTargetCreated,ShotInBoxCreated,CloseShotCreated,TotalShotCreated,HeadersCreated,CreatedCentre,CreatedLeft,CreatedRight
PlayerMatchesDetailID,,,,,,,,,,,,,,,,,,,,,
1,46605,2019-08-09,Home,Alisson,1,90,Liverpool,Norwich City,3,0,...,0,0,0,0,0,0,0,0,0,0
2,46605,2019-08-09,Home,Virgil van Dijk,1,90,Liverpool,Norwich City,3,1,...,0,0,0,0,0,0,0,0,0,0
3,46605,2019-08-09,Home,Joseph Gomez,1,90,Liverpool,Norwich City,3,0,...,0,0,0,0,0,1,0,0,0,0


## Team Data

Let's first create a series of functions that will produce a look up table, allowing us to get data for each team's form as at a given gameweek in the season.

In [11]:
def gameweek_fixtures(gameweeks):
    '''
    Takes a single gameweek, or list of gameweeks, and returns
    a dataframe of games from those gameweeks
    '''
    #Ensure we have a list
    if type(gameweeks)!=list:
        gameweeks=[gameweeks]
    
    return df_melt.loc[df_melt['GameWeek'].isin(gameweeks)]


def last_x_fixtures(team, gameweek, x, home=['Home','Away']):
    '''
    Takes a gameweek, a number of weeks to go back,
    and whether or not we need home matches or away matches.
    Returns a list of matchIDs that fit this criteria
    '''
    #Make sure that home criteria is a list
    if type(home)!=list:
        home = [home]
    
    #Get a dataframe of the team's games before the
    #gameweek matching the home/away criteria
    df_temp = df_melt.loc[(df_melt['GameWeek']<gameweek)
                         &(df_melt['Team']==team)
                         &(df_melt['Home'].isin(home))]
    
    #Take the x most recent
    df_temp = df_temp.sort_values('Date', ascending=False).head(x)
    
    #Return a list of othe match_ids
    return list(df_temp['MatchID'])


def last_x_team_def_stats(team, gameweek, x, home=['Home','Away']):
    '''
    Takes a gameweek, a number of weeks to go back,
    and whether or not we need home matches or away matches.
    Returns a dataframe of the team's average defensive performance in those games
    '''
    #Declare a list of KPIs that we want to extract for the opposing team
    team_kpis = ['RelativeStrength', 'Possession', 'GoalsConceded',
                 'ShotsConceded', 'Touches', 'Passes', 'Tackles',
                 'Clearances', 'Corners', 'Offsides','ShotsInBoxConceded',
                 'CloseShotsConceded', 'HeadersConceded', 'ShotsCentreConceded',
                 'ShotsLeftConceded','ShotsRightConceded']
    
    #get a list of the match IDs that we need
    required_ids = last_x_fixtures(team, gameweek, x, home)
    
    #Create a dataframe of the team performance in those matches
    temp_df = df_matches_t.loc[(df_matches_t['MatchID'].isin(required_ids))
                              &(df_matches_t['ForTeam']==team)]
    
    #Keep the kpis we need
    temp_df = temp_df[team_kpis]
    
    #Create the mean performance across games
    temp_df = pd.DataFrame(temp_df.mean()).T
    
    return temp_df


def team_def_stat_generator(gameweek, total_x=4, home_x=4):
    '''
    Takes a gameweek and outputs a dataframe of 'team defensive performances
    in the last total_x games, and home_x games as appropriate'
    '''
    #Strip down to rows, and create a multi-level index
    #looking at team, then home/away (in case of double game weeks)
    df_temp = gameweek_fixtures(gameweek)[['Team','Home','GameWeek']]
    df_temp = df_temp.groupby(['Team','Home']).mean()
    
    #Instantiate a team 'last total games' dataframe
    df_total_games = pd.DataFrame()

    #For each team/home-away combination in this week... 
    for row in df_temp.itertuples():
        
        #...get the team and home/away status
        team = getattr(row,'Index')[0]
        
        #create the new row for that team
        new_row = last_x_team_def_stats(team, gameweek, total_x)
        
        #append this to the 'last total games' dataframe
        df_total_games = pd.concat([df_total_games, new_row])
    
    #Rename the columns
    df_total_games.columns = [f'OppLast{total_x}{i}'
                              for i in df_total_games.columns]
    
    #Change the index to match df_temp
    df_total_games.index = df_temp.index
    
    #Join this dataframe horizontally to the index
    df_temp = pd.concat([df_temp, df_total_games], axis=1)
    
    #_____________________________________________#
    
    #Now instantiate a team 'last home games' dataframe
    df_home_games = pd.DataFrame()

    #For each team/home-away combination in this week... 
    for row in df_temp.itertuples():
        
        #...get the team and home/away status
        team = getattr(row,'Index')[0]
        home = getattr(row,'Index')[1]
        
        #create the new row for that team
        new_row = last_x_team_def_stats(team, gameweek, home_x, home=home)
        
        #append this to the 'last total games' dataframe
        df_home_games = pd.concat([df_home_games, new_row])
    
    #Rename the columns
    df_home_games.columns = [f'OppLast{home_x}HA{i}'
                             for i in df_home_games.columns]
    
    #Change the index to match df_temp
    df_home_games.index = df_temp.index
    
    #Join this dataframe horizontally to the existing dataframe and return
    df_temp = pd.concat([df_temp, df_home_games], axis=1)
    
    #Put gameweek into the index
    df_temp = df_temp.reset_index(drop=False).set_index(['Team','GameWeek','Home'])
    
    return df_temp



def all_team_def_stat_generator(gameweeks, total_x=4, home_x=4):
    '''
    Takes a list of gameweeks and outputs a dataframe of team performances
    across those gameweeks in the last total_x games, and home_x games as appropriate'
    '''
    
    #Create an empty dataframe
    df_temp = pd.DataFrame()
    
    for week in gameweeks:
        new_rows = team_def_stat_generator(week, total_x=total_x, home_x=home_x)
        df_temp = pd.concat([df_temp, new_rows])
        
    return df_temp

Let's try this out...

(Commented out after running once)

In [12]:
last_gw = max(df_matches_t['GameWeek'])
df_team_def_stats = all_team_def_stat_generator(list(range(4,last_gw+1)))

Great, this seems to have worked! Let's pickle so we don't have to run again...

In [13]:
# Let's pickle this so we don't need to run it again...
with open('Data/df_team_def_stats.pickle', 'wb') as f:
    pickle.dump(df_team_def_stats, f, pickle.HIGHEST_PROTOCOL)

In [14]:
# And opening the pickle...   
with open('Data/df_team_def_stats.pickle', 'rb') as f:
    df_team_def_stats = pickle.load(f)
    
print(df_team_def_stats.shape)
df_team_def_stats.head()

(499, 32)


,,,OppLast4RelativeStrength,OppLast4Possession,OppLast4GoalsConceded,OppLast4ShotsConceded,OppLast4Touches,OppLast4Passes,OppLast4Tackles,OppLast4Clearances,OppLast4Corners,OppLast4Offsides,...,OppLast4HATackles,OppLast4HAClearances,OppLast4HACorners,OppLast4HAOffsides,OppLast4HAShotsInBoxConceded,OppLast4HACloseShotsConceded,OppLast4HAHeadersConceded,OppLast4HAShotsCentreConceded,OppLast4HAShotsLeftConceded,OppLast4HAShotsRightConceded
Team,GameWeek,Home,,,,,,,,,,,,,,,,,,,,,
Arsenal,4,Home,0.333333,59.133333,1.333333,17.333333,762.666667,553.666667,13.000000,25.666667,5.666667,4.000000,...,12.0,28.0,10.0,5.0,11.0,5.0,8.0,6.0,6.0,0.0
Aston Villa,4,Away,-1.333333,42.566667,1.666667,18.333333,551.000000,349.000000,16.000000,26.333333,3.333333,4.333333,...,29.0,39.0,0.0,0.0,16.0,9.0,4.0,8.0,4.0,5.0
Bournemouth,4,Away,-0.333333,38.833333,1.666667,15.666667,519.333333,343.000000,13.666667,20.000000,4.000000,1.666667,...,15.0,25.0,5.0,2.0,11.0,10.0,5.0,9.0,2.0,0.0
Brighton and Hove Albion,4,Away,0.333333,53.266667,1.000000,10.333333,650.000000,444.333333,20.000000,16.666667,6.000000,2.000000,...,24.0,24.0,2.0,3.0,4.0,3.0,1.0,4.0,0.0,0.0
Burnley,4,Home,0.000000,37.966667,1.000000,14.333333,488.333333,290.666667,16.333333,21.666667,4.000000,2.333333,...,21.0,13.0,2.0,5.0,5.0,3.0,0.0,1.0,4.0,0.0


We now want to do something similar with offensive stats (for overall team offence)...

In [15]:
def last_x_team_att_stats(team, gameweek, x, home=['Home','Away']):
    '''
    Takes a gameweek, a number of weeks to go back,
    and whether or not we need home matches or away matches.
    Returns a dataframe of the team's average attacking performance in those games
    '''
    #Declare a list of KPIs that we want to extract for the opposing team
    team_kpis = ['RelativeStrength', 'Possession', 'Goals',
                 'TotalShots', 'Touches', 'Passes', 'Tackles',
                 'Clearances', 'Corners', 'Offsides', 'YellowCards',
                 'FoulsConceded', 'ShotsInBox', 'CloseShots',
                 'Headers', 'ShotsCentre', 'ShotsLeft', 'ShotsRight']
    
    #get a list of the match IDs that we need
    required_ids = last_x_fixtures(team, gameweek, x, home)
    
    #Create a dataframe of the team performance in those matches
    temp_df = df_matches_t.loc[(df_matches_t['MatchID'].isin(required_ids))
                              &(df_matches_t['ForTeam']==team)]
    
    #Keep the kpis we need
    temp_df = temp_df[team_kpis]
    
    #Create the mean performance across games
    temp_df = pd.DataFrame(temp_df.mean()).T
    
    return temp_df


def team_att_stat_generator(gameweek, total_x=4, home_x=4):
    '''
    Takes a gameweek and outputs a dataframe of 'team defensive performances
    in the last total_x games, and home_x games as appropriate'
    '''
    #Strip down to rows, and create a multi-level index
    #looking at team, then home/away (in case of double game weeks)
    df_temp = gameweek_fixtures(gameweek)[['Team','Home','GameWeek']]
    df_temp = df_temp.groupby(['Team','Home']).mean()
    
    #Instantiate a team 'last total games' dataframe
    df_total_games = pd.DataFrame()

    #For each team/home-away combination in this week... 
    for row in df_temp.itertuples():
        
        #...get the team and home/away status
        team = getattr(row,'Index')[0]
        
        #create the new row for that team
        new_row = last_x_team_att_stats(team, gameweek, total_x)
        
        #append this to the 'last total games' dataframe
        df_total_games = pd.concat([df_total_games, new_row])
    
    #Rename the columns
    df_total_games.columns = [f'Last{total_x}{i}'
                              for i in df_total_games.columns]
    
    #Change the index to match df_temp
    df_total_games.index = df_temp.index
    
    #Join this dataframe horizontally to the index
    df_temp = pd.concat([df_temp, df_total_games], axis=1)
    
    #_____________________________________________#
    
    #Now instantiate a team 'last home games' dataframe
    df_home_games = pd.DataFrame()

    #For each team/home-away combination in this week... 
    for row in df_temp.itertuples():
        
        #...get the team and home/away status
        team = getattr(row,'Index')[0]
        home = getattr(row,'Index')[1]
        
        #create the new row for that team
        new_row = last_x_team_att_stats(team, gameweek, home_x, home=home)
        
        #append this to the 'last total games' dataframe
        df_home_games = pd.concat([df_home_games, new_row])
    
    #Rename the columns
    df_home_games.columns = [f'Last{home_x}HA{i}'
                             for i in df_home_games.columns]
    
    #Change the index to match df_temp
    df_home_games.index = df_temp.index
    
    #Join this dataframe horizontally to the existing dataframe and return
    df_temp = pd.concat([df_temp, df_home_games], axis=1)
    
    #Put gameweek into the index
    df_temp = df_temp.reset_index(drop=False).set_index(['Team','GameWeek','Home'])
    
    return df_temp



def all_team_att_stat_generator(gameweeks, total_x=4, home_x=4):
    '''
    Takes a list of gameweeks and outputs a dataframe of team attacking performances
    across those gameweeks in the last total_x games, and home_x games as appropriate'
    '''
    
    #Create an empty dataframe
    df_temp = pd.DataFrame()
    
    for week in gameweeks:
        new_rows = team_att_stat_generator(week, total_x=total_x, home_x=home_x)
        df_temp = pd.concat([df_temp, new_rows])
        
    return df_temp

In [16]:
df_team_att_stats = all_team_att_stat_generator(list(range(4,last_gw+1)))

In [17]:
# Let's pickle this so we don't need to run it again...
with open('Data/df_team_att_stats.pickle', 'wb') as f:
    pickle.dump(df_team_att_stats, f, pickle.HIGHEST_PROTOCOL)

In [18]:
# And opening the pickle...   
with open('Data/df_team_att_stats.pickle', 'rb') as f:
    df_team_att_stats = pickle.load(f)
    
print(df_team_att_stats.shape)
df_team_att_stats.head()

(499, 36)


,,,Last4RelativeStrength,Last4Possession,Last4Goals,Last4TotalShots,Last4Touches,Last4Passes,Last4Tackles,Last4Clearances,Last4Corners,Last4Offsides,...,Last4HACorners,Last4HAOffsides,Last4HAYellowCards,Last4HAFoulsConceded,Last4HAShotsInBox,Last4HACloseShots,Last4HAHeaders,Last4HAShotsCentre,Last4HAShotsLeft,Last4HAShotsRight
Team,GameWeek,Home,,,,,,,,,,,,,,,,,,,,,
Arsenal,4,Home,0.333333,59.133333,1.333333,10.666667,762.666667,553.666667,13.000000,25.666667,5.666667,4.000000,...,10.0,5.0,2.0,13.0,8.0,10.0,1.0,4.0,2.0,2.0
Aston Villa,4,Away,-1.333333,42.566667,1.333333,11.666667,551.000000,349.000000,16.000000,26.333333,3.333333,4.333333,...,0.0,0.0,0.0,9.0,4.0,5.0,1.0,4.0,0.0,0.0
Bournemouth,4,Away,-0.333333,38.833333,1.333333,11.666667,519.333333,343.000000,13.666667,20.000000,4.000000,1.666667,...,5.0,2.0,2.0,13.0,7.0,4.0,2.0,2.0,2.0,3.0
Brighton and Hove Albion,4,Away,0.333333,53.266667,1.333333,11.000000,650.000000,444.333333,20.000000,16.666667,6.000000,2.000000,...,2.0,3.0,1.0,11.0,2.0,3.0,0.0,2.0,1.0,0.0
Burnley,4,Home,0.000000,37.966667,1.666667,13.666667,488.333333,290.666667,16.333333,21.666667,4.000000,2.333333,...,2.0,5.0,0.0,6.0,7.0,5.0,2.0,3.0,1.0,4.0


## Player Data

We now need something similar for players, for which we can probably take a similar approach.

Firstly, we need to create target values. For this, we can use the expected goals measure that we created in the previous notebook.

In [19]:
df_xg = sql('select * from PlayersXG', c).drop('TableIndex', axis=1)
df_xg.head()

,MatchID,Player,GameWeek,ForTeam,AgainstTeam,XG,XA,XGI
PlayersXGID,,,,,,,,
1,46605,Alisson,1,Liverpool,Norwich City,0.000000,0.000000,0.000000
2,46605,Virgil van Dijk,1,Liverpool,Norwich City,0.225926,0.000000,0.225926
3,46605,Joseph Gomez,1,Liverpool,Norwich City,0.000000,0.034483,0.034483
4,46605,Andrew Robertson,1,Liverpool,Norwich City,0.096160,0.077596,0.173756
5,46605,Trent Alexander-Arnold,1,Liverpool,Norwich City,0.121692,1.112207,1.233899


In [20]:
#Add these targets into the player matches dataframe

def XGLookup(match_id, player, metric):
    '''
    Takes a matchid, player, and metric (XG, XA, XGI),
    and returns the metric of the player in the game
    '''
    return df_xg.loc[(df_xg['Player']==player)
                    &(df_xg['MatchID']==int(match_id)),metric].item()


#Apply the function to insert the expected goal stats
df_matches_p['XG'] = df_matches_p.apply(lambda x: XGLookup(x['MatchID'],
                                                    x['Player'],
                                                    'XG'), axis=1)

df_matches_p['XA'] = df_matches_p.apply(lambda x: XGLookup(x['MatchID'],
                                                    x['Player'],
                                                    'XA'), axis=1)

df_matches_p['XGI'] = df_matches_p.apply(lambda x: XGLookup(x['MatchID'],
                                                    x['Player'],
                                                    'XGI'), axis=1)
df_matches_p.head()

,MatchID,Date,Home,Player,GameWeek,Minutes,ForTeam,AgainstTeam,RelativeStrength,Goals,...,ShotInBoxCreated,CloseShotCreated,TotalShotCreated,HeadersCreated,CreatedCentre,CreatedLeft,CreatedRight,XG,XA,XGI
PlayerMatchesDetailID,,,,,,,,,,,,,,,,,,,,,
1,46605,2019-08-09,Home,Alisson,1,90,Liverpool,Norwich City,3,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
2,46605,2019-08-09,Home,Virgil van Dijk,1,90,Liverpool,Norwich City,3,1,...,0,0,0,0,0,0,0,0.225926,0.000000,0.225926
3,46605,2019-08-09,Home,Joseph Gomez,1,90,Liverpool,Norwich City,3,0,...,0,0,1,0,0,0,0,0.000000,0.034483,0.034483
4,46605,2019-08-09,Home,Andrew Robertson,1,90,Liverpool,Norwich City,3,0,...,1,0,1,0,0,1,0,0.096160,0.077596,0.173756
5,46605,2019-08-09,Home,Trent Alexander-Arnold,1,90,Liverpool,Norwich City,3,0,...,2,3,3,1,2,0,0,0.121692,1.112207,1.233899


In [21]:
df_matches_p.head()

,MatchID,Date,Home,Player,GameWeek,Minutes,ForTeam,AgainstTeam,RelativeStrength,Goals,...,ShotInBoxCreated,CloseShotCreated,TotalShotCreated,HeadersCreated,CreatedCentre,CreatedLeft,CreatedRight,XG,XA,XGI
PlayerMatchesDetailID,,,,,,,,,,,,,,,,,,,,,
1,46605,2019-08-09,Home,Alisson,1,90,Liverpool,Norwich City,3,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
2,46605,2019-08-09,Home,Virgil van Dijk,1,90,Liverpool,Norwich City,3,1,...,0,0,0,0,0,0,0,0.225926,0.000000,0.225926
3,46605,2019-08-09,Home,Joseph Gomez,1,90,Liverpool,Norwich City,3,0,...,0,0,1,0,0,0,0,0.000000,0.034483,0.034483
4,46605,2019-08-09,Home,Andrew Robertson,1,90,Liverpool,Norwich City,3,0,...,1,0,1,0,0,1,0,0.096160,0.077596,0.173756
5,46605,2019-08-09,Home,Trent Alexander-Arnold,1,90,Liverpool,Norwich City,3,0,...,2,3,3,1,2,0,0,0.121692,1.112207,1.233899


We now need some kind of index that we'll turn into our master player performance dataframe.

We should keep players that play at least half an hour in each match. We can also get rid of goalkeepers, who are not going to score/assist, and will simply cause class imbalance issues.

In [22]:
#Bring in the players table from the SQL database.
df_players = sql('select * from PlayersBasic', c)

#Find the goalkeepers and defenders
gkdef = list(df_players.loc[df_players['Position'].isin(['GKP'])]['CommentName'])

#Eliminate players playing less than 60mins in a match and goalkeepers
df_60plus = df_matches_p.loc[(df_matches_p['Minutes']>=60)
                            &(~df_matches_p['Player'].isin(gkdef))]

#Create versions of XG, etc. that account for minutes played
for i in ['XG','XA','XGI']:
    df_60plus[f'{i}90'] = 90*df_60plus[i]/df_60plus['Minutes']

df_60plus.head()

,MatchID,Date,Home,Player,GameWeek,Minutes,ForTeam,AgainstTeam,RelativeStrength,Goals,...,HeadersCreated,CreatedCentre,CreatedLeft,CreatedRight,XG,XA,XGI,XG90,XA90,XGI90
PlayerMatchesDetailID,,,,,,,,,,,,,,,,,,,,,
2,46605,2019-08-09,Home,Virgil van Dijk,1,90,Liverpool,Norwich City,3,1,...,0,0,0,0,0.225926,0.000000,0.225926,0.225926,0.000000,0.225926
3,46605,2019-08-09,Home,Joseph Gomez,1,90,Liverpool,Norwich City,3,0,...,0,0,0,0,0.000000,0.034483,0.034483,0.000000,0.034483,0.034483
4,46605,2019-08-09,Home,Andrew Robertson,1,90,Liverpool,Norwich City,3,0,...,0,0,1,0,0.096160,0.077596,0.173756,0.096160,0.077596,0.173756
5,46605,2019-08-09,Home,Trent Alexander-Arnold,1,90,Liverpool,Norwich City,3,0,...,1,2,0,0,0.121692,1.112207,1.233899,0.121692,1.112207,1.233899
6,46605,2019-08-09,Home,Fabinho,1,90,Liverpool,Norwich City,3,0,...,0,0,0,1,0.087209,0.077596,0.164805,0.087209,0.077596,0.164805


Let's now create the functions that will populate our player data.

In [23]:
def last_x_player_fixtures(player, gameweek, x, home=['Home','Away']):
    '''
    Takes a gameweek, a number of weeks to go back,
    and whether or not we need home matches or away matches.
    Returns a list of matchIDs that fit this criteria
    '''
    #Make sure that home criteria is a list
    if type(home)!=list:
        home = [home]
    
    #Get a dataframe of the players's games before the
    #gameweek matching the home/away criteria
    df_temp = df_60plus.loc[(df_60plus['GameWeek']<gameweek)
                         &(df_60plus['Player']==player)
                         &(df_60plus['Home'].isin(home))]
    
    #Take the x most recent
    df_temp = df_temp.sort_values('Date', ascending=False).head(x)
    
    #Return a list of othe match_ids
    return list(df_temp['MatchID'])



def last_x_player_stats(player, gameweek, x,
                        home=['Home','Away'],
                        player_matches_data=df_60plus):
    '''
    Takes a gameweek, a number of weeks to go back (x),
    and whether or not we need home matches or away matches.
    Returns a dataframe of a player's average offensive performance in those games
    '''
    #Declare a list of KPIs that we want to extract for the opposing team
    player_kpis = ['Minutes', 'RelativeStrength', 'Goals', 'ShotsOnTarget',
                   'ShotsInBox', 'CloseShots', 'TotalShots', 'ShotsCentre',
                   'ShotsLeft', 'ShotsRight', 'ShotOnTargetCreated',
                   'ShotInBoxCreated', 'CloseShotCreated', 'TotalShotCreated',
                   'CreatedCentre', 'CreatedLeft', 'CreatedRight']
    
    #get a list of the match IDs that we need
    required_ids = last_x_player_fixtures(player, gameweek, x, home)
    
    #Create a dataframe of the player performance in those matches
    temp_df = player_matches_data.loc[(df_matches_p['MatchID'].isin(required_ids))
                                     &(df_matches_p['Player']==player)]
        
    #Keep the kpis we need and record the length of the dataframe
    temp_df = temp_df[player_kpis]
    matches_covered = len(temp_df)
    
    #Sum the performances across games
    temp_df = pd.DataFrame(temp_df.sum()).T
        
    #Divide the totals by the number of minutes
    #and multiply through by 90*number of games in dataset
    #to get the stats 'per full game'
    for i in player_kpis[2:]:
        temp_df[i] = x*90*temp_df[i]/temp_df['Minutes']
    
    #Get the average 'Relative strength'
    temp_df['RelativeStrength'] = temp_df['RelativeStrength'] / matches_covered
    
    return temp_df



def player_stat_generator(gameweek, total_x=4, home_x=3):
    '''
    Takes a gameweek and outputs a dataframe of player performances
    in the last total_x games, and home_x games as appropriate'
    '''
   
    #Get the dataframe of players in the specified gameweek    
    #Strip down to columns we need
    df_temp = df_60plus.loc[df_60plus['GameWeek']==gameweek,
                           ['Player','GameWeek','Home','MatchID',
                            'XG','XA','XGI','XG90','XA90','XGI90']]
    
    #1) Look up the last game stats
    #Instantiate a player 'last game' dataframe
    df_last_game = pd.DataFrame()
    
    #For each player in the dataframe
    for row in df_temp.itertuples():
        #Get the player name
        player = row.Player
        #create the new row for that player
        new_row = last_x_player_stats(player, gameweek, x=1)
        #append this to the 'last game' dataframe
        df_last_game = pd.concat([df_last_game, new_row])
        
    #Rename the columns
    df_last_game.columns = [f'Last{i}'
                            for i in df_last_game.columns]
    
    #Change the index to match df_temp
    df_last_game.index = df_temp.index
    
    #Join this dataframe horizontally to the index
    df_temp = pd.concat([df_temp, df_last_game], axis=1)
    
    #_____________________________________________

    
    #2) Look up the last x total game stats
    #Instantiate a player 'last game' dataframe
    df_total_games = pd.DataFrame()
    
    #For each player in the dataframe
    for row in df_temp.itertuples():
        #Get the player name
        player = row.Player
        #create the new row for that player
        new_row = last_x_player_stats(player, gameweek, total_x)
        #append this to the 'last game' dataframe
        df_total_games = pd.concat([df_total_games, new_row])
        
    #Rename the columns
    df_total_games.columns = [f'Last{total_x}{i}'
                            for i in df_total_games.columns]
    
    #Change the index to match df_temp
    df_total_games.index = df_temp.index
    
    #Join this dataframe horizontally to the index
    df_temp = pd.concat([df_temp, df_total_games], axis=1)
    
    #_____________________________________________
            
    
    #3) Look up the last x home/away game stats
    #Instantiate a player 'last game' dataframe
    df_home_games = pd.DataFrame()
    
    #For each player in the dataframe
    for row in df_temp.itertuples():
        #Get the player name
        player = row.Player
        home = row.Home
        #create the new row for that player
        new_row = last_x_player_stats(player, gameweek, home_x, home=home)
        #append this to the 'last game' dataframe
        df_home_games = pd.concat([df_home_games, new_row])
        
    #Rename the columns
    df_home_games.columns = [f'Last{home_x}HA{i}'
                            for i in df_home_games.columns]
    
    #Change the index to match df_temp
    df_home_games.index = df_temp.index
    
    #Join this dataframe horizontally to the index
    df_temp = pd.concat([df_temp, df_home_games], axis=1)
    
    #Put gameweek into the index
    df_temp = df_temp.reset_index(drop=False).set_index(
        ['Player','GameWeek','Home','MatchID'])
    
    #Delete old index
    df_temp.drop('PlayerMatchesDetailID', inplace=True, axis=1)
    
    #Drop NAs
    df_temp.dropna(inplace=True)
    
    return df_temp


def all_player_stat_generator(gameweeks, total_x=4, home_x=4):
    '''
    Takes a list of gameweeks and outputs a dataframe of team performances
    across those gameweeks in the last total_x games, and home_x games as appropriate'
    '''
    
    #Create an empty dataframe
    df_temp = pd.DataFrame()
    
    for week in gameweeks:
        new_rows = player_stat_generator(week, total_x=total_x, home_x=home_x)
        df_temp = pd.concat([df_temp, new_rows])
        
    return df_temp

Let's use these functions to create the dataframe of all player's stats.

(Comment out, once run once...)

In [24]:
last_gw = max(df_matches_t['GameWeek'])
df_player_stats = all_player_stat_generator(list(range(4,last_gw+1)))

Let's pickle this...

In [25]:
# Let's pickle this so we don't need to run it again...
with open('Data/df_player_stats.pickle', 'wb') as f:
    pickle.dump(df_player_stats, f, pickle.HIGHEST_PROTOCOL)

In [26]:
# And opening the pickle...   
with open('Data/df_player_stats.pickle', 'rb') as f:
    df_player_stats = pickle.load(f)
    
print(df_player_stats.shape)
df_player_stats.head()

(4493, 57)


,,,,XG,XA,XGI,XG90,XA90,XGI90,LastMinutes,LastRelativeStrength,LastGoals,LastShotsOnTarget,...,Last4HAShotsCentre,Last4HAShotsLeft,Last4HAShotsRight,Last4HAShotOnTargetCreated,Last4HAShotInBoxCreated,Last4HACloseShotCreated,Last4HATotalShotCreated,Last4HACreatedCentre,Last4HACreatedLeft,Last4HACreatedRight
Player,GameWeek,Home,MatchID,,,,,,,,,,,,,,,,,,,,,
Sokratis,4,Home,46635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,90,-1.0,0.000000,0.000000,...,0.000000,0.0,0.000000,4.000000,4.0,4.000000,4.000000,4.0,0.0,0.0
Ainsley Maitland-Niles,4,Home,46635,0.018564,0.773981,0.792545,0.018564,0.773981,0.792545,90,-1.0,0.000000,0.000000,...,0.000000,0.0,0.000000,4.000000,0.0,4.000000,4.000000,0.0,0.0,0.0
David Luiz,4,Home,46635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,90,-1.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
Matteo Guendouzi,4,Home,46635,0.460586,0.426104,0.886690,0.460586,0.426104,0.886690,86,-1.0,0.000000,0.000000,...,0.000000,0.0,4.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
Alexandre Lacazette,4,Home,46635,0.890046,0.211864,1.101910,0.890046,0.211864,1.101910,71,1.0,1.267606,2.535211,...,5.070423,0.0,5.070423,5.070423,0.0,5.070423,5.070423,0.0,0.0,0.0


## Linking the dataframes

We now need to do is link each row in the the player stats table to the appropriate row in the team stats table.

Let's create a function that takes a match ID, and a 'home/away' status, and returns the name of the opposing team. 

In [27]:
def opponent_lookup(match, home):
    '''
    Takes a MatchID and whether or not the player was playing
    at home. Returns the name of the opposing team.
    '''
    return df_melt.loc[(df_melt['MatchID']==match)
           &(df_melt['Home']!=home),'Team'].item()


def home_flipper(x):
    '''
    Takes either 'Home' or 'Away'
    Returns the opposite as a string
    '''
    
    if x == 'Home':
        return 'Away'
    else:
        return 'Home'


def player_stats_extender(player_df, team_df=df_team_def_stats):
    '''
    Takes a player_stats dataframe, and extends it with
    the appropriate opposition team data, taken from the team_df.
    '''
    
    #This will work by creating a seperate dataframe of teams data
    #and then concating it with the existing dataframe
    
    #Create a blank dataframe
    df_temp = pd.DataFrame()
    
    #For each row in the player dataframe..
    for row in player_df.itertuples():
        #Get the gameweek and the home status
        gameweek = getattr(row,'Index')[1]
        home = getattr(row,'Index')[2]
        match_id = getattr(row,'Index')[3]
        
        #Get the opposition team
        against_team = opponent_lookup(match_id, home)
        against_home = home_flipper(home)
        
        #Get the opposition team stats for that match
        new_row = team_df.loc[[(against_team, gameweek, against_home)]]
        
        #Append this row to the bottom of the blank dataframe
        df_temp = pd.concat([df_temp, new_row])
        
    #We now set the index of the new df equal to the player df
    df_temp.index = player_df.index
    
    #Now merge them together
    new_df = pd.concat([player_df, df_temp], axis=1)
    
    return new_df

In [28]:
df_full = player_stats_extender(df_player_stats)
df_full.head()

,,,,XG,XA,XGI,XG90,XA90,XGI90,LastMinutes,LastRelativeStrength,LastGoals,LastShotsOnTarget,...,OppLast4HATackles,OppLast4HAClearances,OppLast4HACorners,OppLast4HAOffsides,OppLast4HAShotsInBoxConceded,OppLast4HACloseShotsConceded,OppLast4HAHeadersConceded,OppLast4HAShotsCentreConceded,OppLast4HAShotsLeftConceded,OppLast4HAShotsRightConceded
Player,GameWeek,Home,MatchID,,,,,,,,,,,,,,,,,,,,,
Sokratis,4,Home,46635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,90,-1.0,0.000000,0.000000,...,17.0,23.0,2.0,1.0,22.0,14.0,5.0,11.0,6.0,5.0
Ainsley Maitland-Niles,4,Home,46635,0.018564,0.773981,0.792545,0.018564,0.773981,0.792545,90,-1.0,0.000000,0.000000,...,17.0,23.0,2.0,1.0,22.0,14.0,5.0,11.0,6.0,5.0
David Luiz,4,Home,46635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,90,-1.0,0.000000,0.000000,...,17.0,23.0,2.0,1.0,22.0,14.0,5.0,11.0,6.0,5.0
Matteo Guendouzi,4,Home,46635,0.460586,0.426104,0.886690,0.460586,0.426104,0.886690,86,-1.0,0.000000,0.000000,...,17.0,23.0,2.0,1.0,22.0,14.0,5.0,11.0,6.0,5.0
Alexandre Lacazette,4,Home,46635,0.890046,0.211864,1.101910,0.890046,0.211864,1.101910,71,1.0,1.267606,2.535211,...,17.0,23.0,2.0,1.0,22.0,14.0,5.0,11.0,6.0,5.0


## Linking the team dataframes

If we want to make a dataframe to help predict clean sheet potential, then we should join team attacking with team defensive datasets.

In [29]:
from datetime import date

def team_stats_extender(team_def_df, team_att_df=df_team_att_stats):
    '''
    Creates a dataframe to predict whether or not a team will keep a clean sheet.
    Takes a team defence dataframe, and extends it with
    the appropriate opposition team offence data, taken from the team_df.
    '''
    
    #This will work by taking the index of the melted fixture dataframe and
    #appending the appropriate defence and opposition offence stats onto it
    
    #Check how many gameweeks of data we have
    played_matches = df_player_stats.reset_index()['MatchID'].unique()
    
    #Create a temporary dataframe that has an index of team, gameweek, home, matchid
    #Make sure we're only including games that have already been played
    new_df = df_melt.loc[(df_melt['GameWeek']>=4)
                        &(df_melt['MatchID'].isin(played_matches))]
    new_df = new_df.set_index(['Team','GameWeek','Home','MatchID'])

    #Create a blank dataframe, which we'll populated with rows
    df_temp = pd.DataFrame()
    
    #For each row in the fixture dataframe..
    for row in new_df.itertuples():
        #Get the gameweek, home status, and matchid
        team = getattr(row,'Index')[0]
        gameweek = getattr(row,'Index')[1]
        home = getattr(row,'Index')[2]
        match_id = getattr(row,'Index')[3]
                
        #Get the 'defence' team stats
        new_row_def = team_def_df.loc[[(team, int(gameweek), home)]]
        
        #Get the opposition team
        against_team = opponent_lookup(match_id, home)
        against_home = home_flipper(home)
        
        #Get the opposition team attacking stats for that match
        new_row_att = team_att_df.loc[[(against_team, int(gameweek), against_home)]]
        
        #Join the rows together
        new_row = pd.concat([new_row_def.reset_index(drop=True),
                             new_row_att.reset_index(drop=True)], axis=1)
        
        #Append this row to the bottom of the blank dataframe
        df_temp = pd.concat([df_temp, new_row])
    
    #We now set the index of the new_rows dataframe equal to the melt dataframe
    df_temp.index = new_df.index
    
    #Now merge them together
    new_df = pd.concat([new_df, df_temp], axis=1)
    
    #Drop various non-important columns that we'll have on the new_df
    new_df.drop(['Date','AgainstTeam',
                 'OppLast4Corners','OppLast4Offsides',
                 'OppLast4HACorners','OppLast4HAOffsides',
                 'Last4Clearances','Last4YellowCards',
                 'Last4FoulsConceded','Last4HAFoulsConceded',
                 'Last4HAClearances','Last4HAYellowCards'],
                axis=1, inplace=True)
    
    #Swap columns around, so that 'opp' prefix makes sense
    new_df.columns = [i.replace('Opp','') if i[:3]=='Opp' else 'Opp'+i
                      for i in new_df.columns]
    
    return new_df

In [30]:
df_team_full = team_stats_extender(df_team_def_stats)
df_team_full.head()

,,,,Last4RelativeStrength,Last4Possession,Last4GoalsConceded,Last4ShotsConceded,Last4Touches,Last4Passes,Last4Tackles,Last4Clearances,Last4ShotsInBoxConceded,Last4CloseShotsConceded,...,OppLast4HAPasses,OppLast4HATackles,OppLast4HACorners,OppLast4HAOffsides,OppLast4HAShotsInBox,OppLast4HACloseShots,OppLast4HAHeaders,OppLast4HAShotsCentre,OppLast4HAShotsLeft,OppLast4HAShotsRight
Team,GameWeek,Home,MatchID,,,,,,,,,,,,,,,,,,,,,
Arsenal,4,Home,46635,0.333333,59.133333,1.333333,17.333333,762.666667,553.666667,13.000000,25.666667,10.000000,5.000000,...,506.0,17.0,2.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0
Tottenham Hotspur,4,Away,46635,0.666667,64.966667,1.333333,15.000000,811.666667,618.666667,14.666667,11.333333,11.000000,7.666667,...,529.0,12.0,10.0,5.0,8.0,10.0,1.0,4.0,2.0,2.0
Burnley,4,Home,46636,0.000000,37.966667,1.000000,14.333333,488.333333,290.666667,16.333333,21.666667,7.000000,6.000000,...,530.0,7.0,9.0,1.0,9.0,8.0,1.0,4.0,3.0,2.0
Liverpool,4,Away,46636,2.333333,57.900000,1.000000,11.666667,714.666667,520.000000,17.666667,15.333333,6.000000,5.333333,...,332.0,21.0,2.0,5.0,7.0,5.0,2.0,3.0,1.0,4.0
Chelsea,4,Home,46637,0.666667,52.466667,2.333333,9.666667,729.333333,506.666667,24.000000,14.333333,4.333333,5.000000,...,428.0,14.0,4.0,1.0,2.0,4.0,0.0,2.0,0.0,0.0


We now need to create a target, i.e. what is the expected goals conceded for each team in each week?

Let's bring this data into the workbook.

In [31]:
df_xgc = sql('select * from TeamsXGC', c).drop('TableIndex', axis=1)
df_xgc.head()

,GameWeek,Team,MatchID,GoalsConceded,XGC
TeamsXGCID,,,,,
1,1,Arsenal,46614,0,0.996303
2,1,Aston Villa,46610,3,3.240594
3,1,Bournemouth,46606,1,1.294230
4,1,Brighton and Hove Albion,46611,0,1.225186
5,1,Burnley,46607,0,1.329473


In [32]:
xgc = []

for row in df_team_full.itertuples():
    team = getattr(row,'Index')[0]
    match_id = getattr(row,'Index')[3]
    
    goals_conceded = df_xgc.loc[(df_xgc['Team']==team)
                               &(df_xgc['MatchID']==int(match_id)),
                                'XGC'].item()
    xgc.append(goals_conceded)

We can commit this to the team dataframe.

In [33]:
df_team_full['XGC'] = xgc

Let's save down these datasets so that we can use them in later workbooks.

In [34]:
df_index_t = df_team_full.reset_index().iloc[:,:4]
df_targets_t = df_team_full.reset_index(drop=True).iloc[:,[-1]]
df_features_t = df_team_full.reset_index(drop=True).iloc[:,:-1]

In [35]:
df_index_t.to_csv('Data/Datasets/df_index_t.csv')
df_targets_t.to_csv('Data/Datasets/df_targets_t.csv')
df_features_t.to_csv('Data/Datasets/df_features_t.csv')

## Using Our Dataset For Modelling

Let's see if there are any especially strong indicators of any of the features...

In [36]:
#Instantiate lists that we'll use for columns later on
targets = []
features = []
correlations = []

#For each feature and each target
for t in list(df_full.columns[:6]):
    for f in list(df_full.columns[6:]):        
        targets.append(t)
        features.append(f)
        t_series = df_full[t]
        f_series = df_full[f]
        #...calculate the correlation
        correlation = np.corrcoef(t_series, f_series)[0][1]
        correlations.append(correlation)
        
df_correlation = pd.DataFrame({'Target':targets,
                               'Feature':features,
                               'Correlation':correlations})

df_correlation.sort_values('Correlation', ascending=False, inplace=True)

In [37]:
df_correlation.loc[df_correlation['Target']=='XG'].head(10)

,Target,Feature,Correlation
23,XG,Last4TotalShots,0.458993
21,XG,Last4ShotsInBox,0.450941
22,XG,Last4CloseShots,0.448485
20,XG,Last4ShotsOnTarget,0.436167
40,XG,Last4HATotalShots,0.425736
38,XG,Last4HAShotsInBox,0.418432
39,XG,Last4HACloseShots,0.410787
24,XG,Last4ShotsCentre,0.400445
37,XG,Last4HAShotsOnTarget,0.381784
19,XG,Last4Goals,0.364305


In [38]:
df_correlation.loc[df_correlation['Target']=='XA'].head(10)

,Target,Feature,Correlation
113,XA,Last4TotalShotCreated,0.310434
130,XA,Last4HATotalShotCreated,0.292247
111,XA,Last4ShotInBoxCreated,0.290071
128,XA,Last4HAShotInBoxCreated,0.270023
112,XA,Last4CloseShotCreated,0.262498
114,XA,Last4CreatedCentre,0.252524
131,XA,Last4HACreatedCentre,0.240232
110,XA,Last4ShotOnTargetCreated,0.236717
129,XA,Last4HACloseShotCreated,0.234154
96,XA,LastTotalShotCreated,0.232457


In [39]:
df_correlation.loc[df_correlation['Target']=='XGI'].head(10)

,Target,Feature,Correlation
189,XGI,Last4TotalShots,0.454285
188,XGI,Last4CloseShots,0.430412
206,XGI,Last4HATotalShots,0.429995
187,XGI,Last4ShotsInBox,0.411789
186,XGI,Last4ShotsOnTarget,0.408840
205,XGI,Last4HACloseShots,0.401211
204,XGI,Last4HAShotsInBox,0.386256
203,XGI,Last4HAShotsOnTarget,0.370981
190,XGI,Last4ShotsCentre,0.345601
185,XGI,Last4Goals,0.333948


In [40]:
df_correlation.loc[df_correlation['Target']=='XG90'].head(10)

,Target,Feature,Correlation
272,XG90,Last4TotalShots,0.464450
270,XG90,Last4ShotsInBox,0.457027
271,XG90,Last4CloseShots,0.451990
269,XG90,Last4ShotsOnTarget,0.439660
289,XG90,Last4HATotalShots,0.430971
287,XG90,Last4HAShotsInBox,0.424090
288,XG90,Last4HACloseShots,0.414620
273,XG90,Last4ShotsCentre,0.404963
286,XG90,Last4HAShotsOnTarget,0.386564
290,XG90,Last4HAShotsCentre,0.366854


In [41]:
df_correlation.loc[df_correlation['Target']=='XA90'].head(10)

,Target,Feature,Correlation
362,XA90,Last4TotalShotCreated,0.313383
379,XA90,Last4HATotalShotCreated,0.295409
360,XA90,Last4ShotInBoxCreated,0.291419
377,XA90,Last4HAShotInBoxCreated,0.272332
361,XA90,Last4CloseShotCreated,0.266671
363,XA90,Last4CreatedCentre,0.251740
359,XA90,Last4ShotOnTargetCreated,0.240507
380,XA90,Last4HACreatedCentre,0.240110
378,XA90,Last4HACloseShotCreated,0.236759
345,XA90,LastTotalShotCreated,0.233802


In [42]:
df_correlation.loc[df_correlation['Target']=='XGI90'].head(10)

,Target,Feature,Correlation
438,XGI90,Last4TotalShots,0.464556
455,XGI90,Last4HATotalShots,0.440135
437,XGI90,Last4CloseShots,0.438967
436,XGI90,Last4ShotsInBox,0.422227
435,XGI90,Last4ShotsOnTarget,0.416488
454,XGI90,Last4HACloseShots,0.410590
453,XGI90,Last4HAShotsInBox,0.395720
452,XGI90,Last4HAShotsOnTarget,0.381217
439,XGI90,Last4ShotsCentre,0.353981
421,XGI90,LastTotalShots,0.337339


So there aren't too many strong correlations one way or the other (as we'd expect with a binary classification problem), though the features that we'd expect to have higher correlations do have them (which is good!)

Let's save these tables down for use in future modelling.

In [43]:
df_index_p = df_full.reset_index().iloc[:,:4]
df_targets_p = df_full.reset_index(drop=True).iloc[:,:6]
df_features_p = df_full.reset_index(drop=True).iloc[:,6:]

In [44]:
df_index_p.to_csv('Data/Datasets/df_index_p.csv')
df_targets_p.to_csv('Data/Datasets/df_targets_p.csv')
df_features_p.to_csv('Data/Datasets/df_features_p.csv')